In [5]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 23.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 KB 111.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 71.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.1/774.1 KB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 84.5 MB/s eta 0:00:0000:0100:01


In [1]:
import time
import torch
from transformers import T5Tokenizer
from vllm.model_executor.models import T5ForVLLM, T5ForVLLMWithCache, T5ForVLLMWithDistributedCache

# Define model and tokenizer paths
model_dir = '/data/data/vllm/vllm/model_executor/models/t5-small-model/'
tokenizer_dir = '/data/data/vllm/vllm/model_executor/models/t5-small-tokenizer/'

# Initialize tokenizers and models
tokenizer = T5Tokenizer.from_pretrained(tokenizer_dir)
basic_model = T5ForVLLM(model_dir, tokenizer_dir)
cached_model = T5ForVLLMWithCache(model_dir, tokenizer_dir)
optimized_model = T5ForVLLMWithDistributedCache(model_dir, tokenizer_dir)

# Input text for testing
input_text = "Translate English to French: Hi How are you, what's happening, tell me something new?"

# Function to benchmark a model
def benchmark_model(model, text, iterations=10):
    # Warm up model
    for _ in range(2):
        model.generate(text)

    # Measure latency and time to first token
    start_time = time.time()
    for _ in range(iterations):
        start = time.time()
        model.generate(text)
        first_token_time = time.time() - start  # This simplifies first token time to full generation time

    total_time = time.time() - start_time
    avg_latency = total_time / iterations
    avg_first_token_time = first_token_time  # Simplistic assumption

    # Throughput calculation
    throughput = iterations / total_time

    return avg_latency, throughput, avg_first_token_time

# Run benchmarks
basic_latency, basic_throughput, basic_first_token = benchmark_model(basic_model, input_text)
cached_latency, cached_throughput, cached_first_token = benchmark_model(cached_model, input_text)
optimized_latency, optimized_throughput, optimized_first_token = benchmark_model(optimized_model, input_text)

# Print results
print("Basic Model - Avg Latency: {:.3f}s, Throughput: {:.2f} req/s, Time to First Token: {:.3f}s".format(basic_latency, basic_throughput, basic_first_token))
print("Cached Model - Avg Latency: {:.3f}s, Throughput: {:.2f} req/s, Time to First Token: {:.3f}s".format(cached_latency, cached_throughput, cached_first_token))
print("Optimized Model - Avg Latency: {:.3f}s, Throughput: {:.2f} req/s, Time to First Token: {:.3f}s".format(optimized_latency, optimized_throughput, optimized_first_token))


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 04-30 03:01:48 pynccl.py:58] Loading nccl from library /root/.config/vllm/nccl/cu12/libnccl.so.2.18.1


2024-04-30 03:01:48,774	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Basic Model - Avg Latency: 1.031s, Throughput: 0.97 req/s, Time to First Token: 0.847s
Cached Model - Avg Latency: 0.002s, Throughput: 459.32 req/s, Time to First Token: 0.002s
Optimized Model - Avg Latency: 0.002s, Throughput: 437.17 req/s, Time to First Token: 0.002s


In [1]:
import time
from transformers import T5Tokenizer
from vllm.model_executor.models import T5ForVLLMWithDistributedCache

# Define model and tokenizer paths
# Define model and tokenizer paths
model_dir = '/data/data/vllm/vllm/model_executor/models/t5-small-model/'
tokenizer_dir = '/data/data/vllm/vllm/model_executor/models/t5-small-tokenizer/'

# Initialize the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained(tokenizer_dir)
model = T5ForVLLMWithDistributedCache(model_dir, tokenizer_dir)

# List of prompts to test
prompts = {
    "exact_match": "Translate English to French: Hello, how are you?",
    "case_change": "translate english to french: hello, how are you?",
    "punctuation_change": "Translate English to French: Hello, how are you!",
    "whitespace_change": "Translate English to French: Hello, how are you? ",
    "minor_text_change": "Translate English to French: Hello, how are you doing?",
    "frequent_request": "What is the weather like in Paris today?"
}

# Function to test each prompt and measure caching impact
def test_caching(model, prompts):
    results = {}
    for prompt_type, text in prompts.items():
        start_time = time.time()
        output, cache_hit = model.generate(text)  # Receive output and cache hit status
        duration = time.time() - start_time

        results[prompt_type] = {
            "duration": duration,
            "cache_hit": cache_hit,
            "output": output
        }
    return results

# Run the test
results = test_caching(model, prompts)

# Print results
for prompt_type, info in results.items():
    cache_status = "Hit" if info["cache_hit"] else "Miss"
    print(f"Prompt Type: {prompt_type}, Cache Status: {cache_status}, Time Taken: {info['duration']:.2f}s, Output: {info['output'][:30]}...")


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 04-30 06:44:28 pynccl.py:58] Loading nccl from library /root/.config/vllm/nccl/cu12/libnccl.so.2.18.1


2024-04-30 06:44:29,334	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Prompt Type: exact_match, Cache Status: Hit, Time Taken: 0.00s, Output: ['Bonjour, comment êtes-vous?']...
Prompt Type: case_change, Cache Status: Miss, Time Taken: 0.93s, Output: ['Hallo, comment êtes-vous?']...
Prompt Type: punctuation_change, Cache Status: Miss, Time Taken: 0.31s, Output: ['Bonjour, comment êtes-vous!']...
Prompt Type: whitespace_change, Cache Status: Hit, Time Taken: 0.00s, Output: ['Bonjour, comment êtes-vous?']...
Prompt Type: minor_text_change, Cache Status: Miss, Time Taken: 0.27s, Output: ['Bonjour, comment faites-vous?']...
Prompt Type: frequent_request, Cache Status: Miss, Time Taken: 0.31s, Output: ['Wie sieht das Wetter in Paris heute?']...
